In [1]:
import pandas as pd
from pydataset import data
import numpy as np
import datetime 
 

In [2]:
from env import host, user, password

In [8]:
mpg = data('mpg')

In [10]:
mpg.head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


In [11]:
# On average, which manufacturer has the best miles per gallon?
mpg['average_mpg'] = mpg[['cty', 'hwy']].mean(axis= 1)
mpg.groupby('manufacturer').average_mpg.mean().idxmax()

'honda'

In [12]:
# How many different manufacturers are there?
len(mpg.groupby('manufacturer'))

15

In [13]:
# How many different models are there?
len(mpg.groupby('model'))

38

In [14]:
# Do automatic or manual cars have better miles per gallon?

auto_avg_mpg = mpg[mpg.trans.str.contains('auto')].average_mpg.mean()
auto_avg_mpg

19.130573248407643

In [15]:
manual_avg_mpg = mpg[mpg.trans.str.contains('manual')].average_mpg.mean()
manual_avg_mpg

22.227272727272727

In [16]:
print(f'Automatic average mpg = {auto_avg_mpg}. Manual averge mpg = {manual_avg_mpg}')

Automatic average mpg = 19.130573248407643. Manual averge mpg = 22.227272727272727


In [17]:
users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users

,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [18]:
roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles

,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


In [19]:
# Copy the users and roles dataframes from the examples above. What do you think a right join would look like?
# An outer join? What happens if you drop the foreign keys from the dataframes and try to merge them?

pd.merge(users, roles, left_on='role_id', right_on='id', how='right')

,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1,admin
1,2.0,joe,2.0,2,author
2,3.0,sally,3.0,3,reviewer
3,4.0,adam,3.0,3,reviewer
4,NaN,NaN,NaN,4,commenter


In [20]:
pd.merge(users, roles, left_on='role_id', right_on='id', how='outer')

,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1.0,admin
1,2.0,joe,2.0,2.0,author
2,3.0,sally,3.0,3.0,reviewer
3,4.0,adam,3.0,3.0,reviewer
4,5.0,jane,NaN,NaN,NaN
5,6.0,mike,NaN,NaN,NaN
6,NaN,NaN,NaN,4.0,commenter


In [30]:
def get_db_url(database):
    from env import host, user, password
    url = f'mysql+pymysql://{user}:{password}@{host}/{database}'
    return url

In [23]:
employees = pd.read_sql("SELECT* from employees", get_db_url('employees'))

In [24]:
employees.describe()

,emp_no
count,300024.000000
mean,253321.763392
std,161828.235540
min,10001.000000
25%,85006.750000
50%,249987.500000
75%,424993.250000
max,499999.000000


In [25]:
titles = pd.read_sql("select * from titles", get_db_url('employees'))

In [26]:
# Visualize the number of employees with each title.
title_employees = titles.groupby('title')

import matplotlib.pyplot as plt
title_employees.agg(['count']).emp_no.plot.bar()
plt.title('Employees with title')
plt.xlabel('Title')
plt.ylabel('count')
plt.xticks(rotation = 45)

(array([0, 1, 2, 3, 4, 5, 6]), <a list of 7 Text xticklabel objects>)

In [27]:
# Join the employees and titles dataframes together.

merge_employees = pd.merge(employees, titles, left_on='emp_no', right_on='emp_no', how='left')
merge_employees

,emp_no,birth_date,first_name,last_name,gender,hire_date,title,from_date,to_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,Senior Engineer,1986-06-26,9999-01-01
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21,Staff,1996-08-03,9999-01-01
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28,Senior Engineer,1995-12-03,9999-01-01
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Engineer,1986-12-01,1995-12-01
4,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Senior Engineer,1995-12-01,9999-01-01
...,...,...,...,...,...,...,...,...,...
443303,499997,1961-08-03,Berhard,Lenart,M,1986-04-21,Engineer,1987-08-30,1992-08-29
443304,499997,1961-08-03,Berhard,Lenart,M,1986-04-21,Senior Engineer,1992-08-29,9999-01-01
443305,499998,1956-09-05,Patricia,Breugel,M,1993-10-13,Senior Staff,1998-12-27,9999-01-01
443306,499998,1956-09-05,Patricia,Breugel,M,1993-10-13,Staff,1993-12-27,1998-12-27


In [28]:
# Visualize how frequently employees change titles.
merged = merge_employees.groupby('emp_no').count()
merged = merged.reset_index()

In [ ]:
merged = merged[['emp_no', 'to_date']]
job_changes = merged.groupby('to_date').count()
job_changes.plot.bar()
plt.title('Number of titles')
plt.xlabel('number of titles per employee')
plt.ylabel('number of employees')
plt.xticks(rotation = 0)
plt.legend('')

In [ ]:
latest_date = datetime.date(2000,1,28)
merge_employees['days_in_job'] = np.where(merge_employees.to_date.astype('str') == '9999-01-01', ((latest_date-merge_employees.from_date)), (merge_employees.to_date - merge_employees.from_date).astype('timedelta64[D]')) 

s_array = merge_employees[['emp_no', "days_in_job"]]
s_array['years_in_job'] = s_array.days_in_job.astype('timedelta64[D]')/365

In [ ]:
new_array = s_array.groupby('emp_no').mean().reset_index()

In [ ]:
new_array

In [ ]:
# For each title, find the hire date of the employee that was hired most recently with that title.
merge_employees.groupby(['title']).hire_date.agg(max)

In [ ]:
# Write the code necessary to create a cross tabulation of the number of titles by department.
# (Hint: this will involve a combination of SQL and python/pandas code)

dept_emp = pd.read_sql("select * from dept_emp", get_db_url('employees'))


In [ ]:
#Selecting current employees only i.e. where to_date = 9999-01-01
dept_emp_current = dept_emp[dept_emp.to_date.astype('str') == '9999-01-01']
dept_emp_current

In [ ]:
#Selecting current titles only i.e. where to_date = 9999-01-01
titles_current = titles[titles.to_date.astype('str') == '9999-01-01']
titles_current

In [ ]:
titles_and_dept = pd.merge(titles, dept_emp, left_on='emp_no', right_on='emp_no', how='outer')

In [ ]:
#used crosstab to obtain cross tablulation of dept and titles
pd.crosstab(titles_and_dept.title, titles_and_dept.dept_no)

In [ ]:
get_db_url('chipotle')

In [ ]:
orders = pd.read_sql("SELECT* from orders", get_db_url('chipotle'))

In [ ]:
orders.head(5)

In [ ]:
orders['price'] = orders.item_price.str.replace("$", "").astype(float) 
orders


In [ ]:
# What is the total price for each order?
orders.groupby('order_id').price.sum()

In [ ]:
# What are the most popular 3 items?

popular = orders.groupby('item_name').id.count()

In [ ]:
popular = popular.reset_index()
popular = popular.rename(columns={'id': 'number_of_orders'})
popular.head(5)

In [ ]:
popular.sort_values(by= 'number_of_orders', ascending = False).head(3)

In [ ]:
# Which item has produced the most revenue?
revenue = orders.groupby('item_name').price.sum()
revenue = revenue.reset_index()
revenue = revenue.rename(columns={'price': 'revenue'})

In [ ]:
revenue.sort_values(by= 'revenue', ascending = False).head(1)